In [ ]:
import pandas as pd

df = pd.read_csv("../../data/raw/train.csv.gz")

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [3]:
import numpy as np

# SalePrice와의 피어슨 상관계수 계산
corr_matrix = df.corr(numeric_only=True)
saleprice_corr = corr_matrix['SalePrice'].sort_values(ascending=False)

print("각 컬럼과 SalePrice의 피어슨 상관계수:")
print(saleprice_corr)


threshold = 0.6
selected_features = saleprice_corr[abs(saleprice_corr) >= threshold].index.tolist()
selected_features.remove('SalePrice')  # 타겟 컬럼은 제외

print("\n모델링에 사용할 feature (상관계수 |0.6| 이상):")
print(selected_features)


각 컬럼과 SalePrice의 피어슨 상관계수:
SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvG

In [16]:
# selected_features의 타입 확인
print("selected_features 타입:", type(selected_features))
print("selected_features:", selected_features)

# 선택된 feature와 타겟 변수만 추출
target = 'SalePrice'
features = selected_features.copy()
features.append(target)
df_selected = train[features].copy()

# 결측치 확인
print("\n각 feature별 결측치 개수:")
print(df_selected.isnull().sum())

# 결측치 처리 (수치형: 평균, 범주형: 최빈값)
for col in selected_features:
    if df_selected[col].dtype == 'object':
        # 범주형: 최빈값으로 대체
        mode = df_selected[col].mode()[0]
        df_selected[col] = df_selected[col].fillna(mode)
    else:
        # 수치형: 평균으로 대체
        mean = df_selected[col].mean()
        df_selected[col] = df_selected[col].fillna(mean)

# 인코딩 (범주형 변수만)
from sklearn.preprocessing import LabelEncoder, StandardScaler

for col in selected_features:
    if df_selected[col].dtype == 'object':
        le = LabelEncoder()
        df_selected[col] = le.fit_transform(df_selected[col])

# 정규화/스케일링 (수치형 변수만)
scaler = StandardScaler()
num_cols = [col for col in selected_features if df_selected[col].dtype != 'object']
df_selected[num_cols] = scaler.fit_transform(df_selected[num_cols])

# 최종 데이터프레임 확인
print("\n전처리된 데이터프레임 info:")
print(df_selected.info())



selected_features 타입: <class 'list'>
selected_features: ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF']

각 feature별 결측치 개수:
OverallQual    0
GrLivArea      0
GarageCars     0
GarageArea     0
TotalBsmtSF    0
1stFlrSF       0
SalePrice      0
dtype: int64

전처리된 데이터프레임 info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   OverallQual  1460 non-null   float64
 1   GrLivArea    1460 non-null   float64
 2   GarageCars   1460 non-null   float64
 3   GarageArea   1460 non-null   float64
 4   TotalBsmtSF  1460 non-null   float64
 5   1stFlrSF     1460 non-null   float64
 6   SalePrice    1460 non-null   int64  
dtypes: float64(6), int64(1)
memory usage: 80.0 KB
None
   OverallQual  GrLivArea  GarageCars  GarageArea  TotalBsmtSF  1stFlrSF  \
0     0.651479   0.370333    0.311725    0.351000    -0.459303 -0.793434 

In [27]:
df_selected.head()

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,SalePrice
0,0.651479,0.370333,0.311725,0.351000,-0.459303,-0.793434,208500
1,-0.071836,-0.482512,0.311725,-0.060731,0.466465,0.257140,181500
2,0.651479,0.515013,0.311725,0.631726,-0.313369,-0.627826,223500
3,0.651479,0.383659,1.650307,0.790804,-0.687324,-0.521734,140000
4,1.374795,1.299326,1.650307,1.698485,0.199680,-0.045611,250000


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# 원본 SalePrice와 log(SalePrice) 모두에 대해 간단한 선형회귀로 성능 비교

X = df_selected.drop('SalePrice', axis=1).values
y = df_selected['SalePrice'].values
y_log = np.log1p(y)

# 원본 타겟
model = LinearRegression()
scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
print("원본 SalePrice RMSE (5-fold CV):", -scores.mean())

# log 변환 타겟
scores_log = cross_val_score(model, X, y_log, cv=5, scoring='neg_root_mean_squared_error')
print("log1p(SalePrice) RMSE (5-fold CV):", -scores_log.mean())


원본 SalePrice RMSE (5-fold CV): 39427.764555321155
log1p(SalePrice) RMSE (5-fold CV): 0.18496175823100316

log 변환을 사용하면 RMSE가 더 낮아지면 성능이 향상된 것임을 의미합니다.


In [28]:
# log 변환된 타겟을 사용할 것이므로, df_selected의 SalePrice를 log1p로 변환하여 업데이트합니다.
df_selected['SalePrice'] = np.log1p(df_selected['SalePrice'])
print("df_selected['SalePrice']가 log1p로 변환되었습니다.")


df_selected['SalePrice']가 log1p로 변환되었습니다.


In [ ]:
import pickle

with open('../../data/processed/df_selected_01.pkl', 'wb') as f:
    pickle.dump(df_selected, f)

print("df_selected가 '../../data/processed/df_selected_01.pkl' 파일로 저장되었습니다.")


df_selected가 'df_selected.pkl' 파일로 저장되었습니다.
